In [1]:
from utils import *
from transformers import AutoTokenizer
from transformers import BertForTokenClassification, AdamW, BertModel, BertConfig
import torch
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm, trange

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
from utilities_check_err import *
from visual_test import *

# 1. load model

In [3]:
dir_valid = 'vlsp2018/val_2018.txt'
MAXLEN = 256
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False,use_fast=False)

In [4]:
IO_tag_values = ['PER','LOC','ORG','MISC','O']
IO_data_valid = process_data_for_BERT(dir_valid, tokenizer, IO_tag_values, MAXLEN, type='IO')

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [6]:
IO_tag2idx = IO_data_valid.tag2idx

In [7]:
#BERT base
IO_model = BertForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(IO_tag2idx),
    output_attentions = False,
    output_hidden_states = False
)
IO_model.load_state_dict(torch.load('IO_BERT_MULTI.pt'), strict=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

<All keys matched successfully>

In [8]:
IO_model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

## 2. Testing

In [9]:
data_true, y_predict = predict_data_train(IO_model, tokenizer, IO_tag_values ,IO_data_valid, 1 ,IO_tag2idx, 256)

In [13]:
DATA = []
Y = []
for i in range(len(IO_data_valid.data)):
    data,predict= predict_data_train(IO_model, tokenizer, IO_tag_values ,IO_data_valid, i ,IO_tag2idx, 256)
    DATA.append(data)
    Y.append(predict)

In [19]:
#CHECK LEN
count = 0
for i in range(len(IO_data_valid.data)):
    if len(DATA[i]) != len(Y[i]):
        print(i,'ERRR')
        break
    else:
        count+=1

In [20]:
'''import pickle
with open('data_true.pkl', 'wb') as f:
    pickle.dump(DATA, f)'''
'''import pickle
with open('y_predict.pkl', 'wb') as f:
    pickle.dump(Y, f)'''

In [22]:
file = open("y_predict.pkl",'rb')
y_predict = pickle.load(file)

file = open("data_true.pkl",'rb')
data = pickle.load(file)

In [23]:
y_predict

[['O',
  'O',
  'O',
  'LOC',
  'LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'O',
  'LOC',
  'LOC',
  'LOC',
  'O',
  'LOC',
  'LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'LOC',
  'LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'LOC',
  'LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'ORG',
  'ORG',
  'ORG',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'PER',
  'PER',
  'PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'LOC',
  'LOC',
  'LOC',
  'O',
  'LOC',
  'LOC',
  'LOC',
  'O',
  'LOC',
  'LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',

In [24]:
data

[[('6', '?', 'O'),
  ('ngư', '?', 'O'),
  ('dân', '?', 'O'),
  ('Quảng', '?', 'LOC'),
  ('Ngãi', '?', 'LOC'),
  ('trên', '?', 'O'),
  ('tàu', '?', 'O'),
  ('cá', '?', 'O'),
  ('bị', '?', 'O'),
  ('nạn', '?', 'O'),
  ('đã', '?', 'O'),
  ('được', '?', 'O'),
  ('cứu', '?', 'O'),
  ('hộ', '?', 'O'),
  ('vào', '?', 'O'),
  ('bờ', '?', 'O'),
  ('an', '?', 'O'),
  ('toàn', '?', 'O')],
 [('Sáng', '?', 'O'),
  ('22', '?', 'O'),
  ('/', '?', 'O'),
  ('9', '?', 'O'),
  (',', '?', 'O'),
  ('Trạm', '?', 'LOC'),
  ('Biên', '?', 'LOC'),
  ('phòng', '?', 'LOC'),
  ('Mũi', '?', 'LOC'),
  ('Tấn', '?', 'LOC'),
  ('(', '?', 'O'),
  ('TP', '?', 'LOC'),
  ('Quy', '?', 'LOC'),
  ('Nhơn', '?', 'LOC'),
  (',', '?', 'O'),
  ('Bình', '?', 'LOC'),
  ('Định', '?', 'LOC'),
  (')', '?', 'O'),
  ('đã', '?', 'O'),
  ('tiếp', '?', 'O'),
  ('nhận', '?', 'O'),
  ('6', '?', 'O'),
  ('ngư', '?', 'O'),
  ('dân', '?', 'O'),
  ('Quảng', '?', 'LOC'),
  ('Ngãi', '?', 'LOC'),
  ('trên', '?', 'O'),
  ('tàu', '?', 'O'),
  ('cá', '